<div class = 'alert alert-info'>

- scp the `3 channels val data transforms` from HPC.
- **DELETE THE CACHED DATA FILES UNDER** `batched_Datasets_3_custom_channels` **FOLDER!!!!**
- batch into arrow datasets of tensors
- load in the 4 labels OVR trained models on the 3 custom channels train data.
- get results and see if any of the classes improved in MSE.

<div class = 'alert alert-info'>

# Change the code below as needed - comment out code from last model evaluation and make new code cells for evaluating your 4 label OVR trained models

Mess around with the code and try to figure out a good scheme for making predictions

First pass:

- If a label is predicted to be a `Nan`, fill the result in the .csv file as a zero
- Take a look at how many labels get predicted as `Nan` versus any other label

- May want to take a different approach moving forward, like biasing labels towards + or - 1 if the predicted label is a `Nan`

Remember; encoding labels as `Nan`s was important in training to remove bias in the fine tuning from over predicting zeros, and the hope was that the model might learn where data points should be missing. That being said, we still need to make a discrete (or floating point) prediction for each class/pathology for each image, and we don't know for sure whether a label will be missing or not in the validation data. So, we should pick a substitute label and try it out, then submit the file(s) to eval.ai and check the performance vs our previous benchmarks and older model runs, as well as your `hacked` submission file.

# Imports

In [1]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
import datasets
import scipy.special as sp

# In this notebook

We will make predicitons on the validation data set for Chexpert using the OVR N hot encoded Google ViT fine tuned models

### Load in the raw logits

One vs. rest, 4 labels predictions (including a model trained only on nans)

In [2]:
logits = torch.load('val_logits_3_custom_channels_OVR_4_labels_tensor.pt')

In [2]:
# logits = torch.load('val_logits_OVR_4_labels_tensor_bicubic.pt')

In [4]:
# logits

In [10]:
# logits = torch.load('logits_tensor_bicubic.pth')

In [5]:
# np_logits = logits.numpy()

In [6]:
# np_logits.shape

In [7]:
# np.save('logits_tensor_bicubic.npy', np_logits)

In [3]:
logits.shape

torch.Size([22596, 4, 9])

### Define functions for getting predictions (floating point and discrete preds)

In [4]:
# Logits need to be stacked into one tensor first, making sure the shape is 
# something like (N, 3, 9) for N images

# Get column wise softmax - ie each row entry for each column
probs = np.array([sp.softmax(logits_tensor, axis=0) for logits_tensor in logits])

In [5]:
probs = torch.tensor(probs)

In [6]:
probs

tensor([[[1.8923e-02, 2.1180e-02, 5.1924e-03,  ..., 1.4795e-03,
          2.1112e-03, 1.9044e-01],
         [2.1094e-02, 4.2890e-02, 6.4449e-04,  ..., 1.2077e-03,
          1.5746e-02, 4.2852e-03],
         [6.7380e-02, 1.5264e-01, 3.0875e-03,  ..., 5.1632e-05,
          3.9723e-03, 1.0310e-02],
         [8.9260e-01, 7.8329e-01, 9.9108e-01,  ..., 9.9726e-01,
          9.7817e-01, 7.9497e-01]],

        [[2.0303e-02, 1.9281e-02, 4.9311e-03,  ..., 3.0132e-03,
          3.1686e-03, 7.0449e-02],
         [2.5266e-02, 4.3364e-02, 9.2979e-04,  ..., 2.4274e-03,
          1.7991e-02, 3.8885e-03],
         [1.0244e-01, 2.1971e-01, 3.8752e-03,  ..., 8.0327e-05,
          5.9150e-03, 8.5839e-03],
         [8.5199e-01, 7.1765e-01, 9.9026e-01,  ..., 9.9448e-01,
          9.7293e-01, 9.1708e-01]],

        [[1.7099e-02, 9.0248e-03, 2.5493e-03,  ..., 1.8598e-04,
          1.3195e-03, 7.9832e-01],
         [1.0181e-02, 1.4667e-02, 1.9042e-04,  ..., 1.3538e-04,
          1.1389e-02, 5.9322e-03],
      

In [7]:
probs[0][:,0].sum()

tensor(1.)

In [8]:
probs.shape

torch.Size([22596, 4, 9])

In [9]:
probs.shape[1]

4

In [15]:
x = 'random_pos'
x.startswith('random')

True

In [17]:
type(x)

str

In [18]:
type(x) == str

True

In [16]:
y = 8
y.startswith('random')

AttributeError: 'int' object has no attribute 'startswith'

In [23]:
np.random.rand()

0.8163876871384602

In [10]:
def get_floating_preds(image_probs, custom_nan_val):
    # Takes in a tensor of softmax probabilites, computed column wise,
    # for a single image.
    # Outputs the floating point labels of values in [-1, 1]
    # depending upon the row index of highest probability for each column.

    # If the row corresponding to negative one is the argmax, scale the softmax
    # value by -1.

    # If the row corresponding to zero is the argmax, return 0.

    # If the row corresponding to positive one is the argmax, return the softmax
    # value.
    
    # If the row corresponding to `Nan` is the argmax, return a custom value
    # (could be constant, like 0, or could be np.random.rand).
    
    """
    Input for `custom_nan_val` can either be:
    - a floating point value or an integer
    - a string, with the name 'random_pos', or 'random_neg', resulting in a float
    value between 0 and +1 or +1 depending upon the string
    """
    
    sign_dict = {
        'pos': 1,
        'neg': -1,
    }

    columns = image_probs.shape[1]
    output_labels = []

    for column in range(columns):
        ind = torch.argmax(image_probs[:, column]) 

        if ind == 0:
            output_labels.append(image_probs[ind, column].numpy() * (1))
        elif ind == 1:
            # May want to adjust this to lower the MSE
            output_labels.append(0)
        elif ind == 2:
            output_labels.append(image_probs[ind, column].numpy() * (-1))   
        elif ind == 3:
            # May want to adjust this to lower the MSE
            if type(custom_nan_val) == str:
                suffix = custom_nan_val.split('_')[1]
                output_labels.append(np.random.rand() * sign_dict[suffix])
            else:
                output_labels.append(custom_nan_val)
            
    return output_labels


def get_discrete_preds(image_probs, custom_nan_val):
    # Takes in a tensor of softmax probabilites, computed column wise,
    # for a single image.
    # Outputs the discrete labels [-1, 0, 1] as well as a val for `Nan`
    # depending upon the row index of highest probability for each column.

    # Make the `custom_nan_val` be a constant float or int
    
    columns = image_probs.shape[1]
    output_labels = []

    for column in range(columns):
        ind = torch.argmax(image_probs[:, column]) 

        if ind == 0:
            output_labels.append(1)
        elif ind == 1:
            output_labels.append(0)
        elif ind == 2:
            output_labels.append(-1)   
        elif ind == 3:
            output_labels.append(custom_nan_val)

    return output_labels

In [11]:
classes = [
    'Cardiomegaly',
    'Enlarged Cardiomediastinum',
    'Fracture',
    'Lung Opacity',
    'No Finding',
    'Pleural Effusion',
    'Pleural Other',
    'Pneumonia',
    'Support Devices',
]

Get a data frame in the shape of number of validation images x 10.
Make sure to get the Id values from the `test.csv` file on the HPC which 
matches the patient ID to the data - it is not just 
np.arange(0, len(val_data), 1), it is a bunch of random integers 

Set `Nan`s to 0 for now

In [12]:
probs[0][:,8].sum()

tensor(1.)

In [13]:
discrete_preds = [get_discrete_preds(prob, custom_nan_val=0) for prob in probs]

floating_preds = [get_floating_preds(prob, custom_nan_val=0) for prob in probs]

In [14]:
np.array(floating_preds).shape

(22596, 9)

In [15]:
np.array(discrete_preds).shape

(22596, 9)

In [35]:
# np.save('discrete_preds_bicubic.npy', np.array(discrete_preds))
# np.save('floating_preds_bicubic.npy', np.array(floating_preds))

In [16]:
discrete_dict = {index: column for index, column in enumerate(np.array(discrete_preds).T)}
discrete_df = pd.DataFrame(discrete_dict)
discrete_df.columns = classes

discrete_df

,Cardiomegaly,Enlarged Cardiomediastinum,Fracture,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Support Devices
0,0,0,0,0,-1,-1,0,0,0
1,0,0,0,0,-1,-1,0,0,0
2,0,0,0,1,-1,0,0,0,1
3,0,0,0,1,-1,0,0,0,1
4,0,0,0,0,-1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
22591,0,0,0,0,-1,0,0,0,1
22592,0,0,0,0,-1,0,0,0,1
22593,0,0,0,1,-1,1,0,0,1
22594,0,0,0,1,-1,1,0,0,1


In [17]:
floating_dict = {index: column for index, column in enumerate(np.array(floating_preds).T)}
floating_df = pd.DataFrame(floating_dict)
floating_df.columns = classes

floating_df

,Cardiomegaly,Enlarged Cardiomediastinum,Fracture,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Support Devices
0,0.0,0.0,0.0,0.000000,-0.811068,-0.457232,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.000000,-0.641785,-0.581890,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.496037,-0.974069,0.000000,0.0,0.0,0.798321
3,0.0,0.0,0.0,0.537415,-0.987535,0.000000,0.0,0.0,0.711532
4,0.0,0.0,0.0,0.000000,-0.950609,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
22591,0.0,0.0,0.0,0.000000,-0.951126,0.000000,0.0,0.0,0.688630
22592,0.0,0.0,0.0,0.000000,-0.974998,0.000000,0.0,0.0,0.530088
22593,0.0,0.0,0.0,0.672733,-0.995999,0.629380,0.0,0.0,0.762317
22594,0.0,0.0,0.0,0.645646,-0.995374,0.493284,0.0,0.0,0.849533


In [18]:
Id = np.load('Id.npy')

discrete_df['Id'] = Id
floating_df['Id'] = Id

In [19]:
discrete_df.head()

,Cardiomegaly,Enlarged Cardiomediastinum,Fracture,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Support Devices,Id
0,0,0,0,0,-1,-1,0,0,0,18
1,0,0,0,0,-1,-1,0,0,0,19
2,0,0,0,1,-1,0,0,0,1,44
3,0,0,0,1,-1,0,0,0,1,45
4,0,0,0,0,-1,0,0,0,0,57


In [20]:
floating_df.head()

,Cardiomegaly,Enlarged Cardiomediastinum,Fracture,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Support Devices,Id
0,0.0,0.0,0.0,0.000000,-0.811068,-0.457232,0.0,0.0,0.000000,18
1,0.0,0.0,0.0,0.000000,-0.641785,-0.581890,0.0,0.0,0.000000,19
2,0.0,0.0,0.0,0.496037,-0.974069,0.000000,0.0,0.0,0.798321,44
3,0.0,0.0,0.0,0.537415,-0.987535,0.000000,0.0,0.0,0.711532,45
4,0.0,0.0,0.0,0.000000,-0.950609,0.000000,0.0,0.0,0.000000,57


In [21]:
discrete_df.to_csv('discrete_preds_3_custom_channels.csv')
floating_df.to_csv('floating_preds_3_custom_channels.csv')

<hr>

<div class = 'alert alert-info'>

# Bicubic data transforms analysis

### Unfortunately, simply training a model on the `Nan` values, and at least just inputting a 0 for the values predicted to bee `Nan`s, did not really change the results of the model for OVR encoding of labels.

The average MSE is still 1.29 (without hacking).

However, the floating point values for the pathology of `pleural effusion` went down significantly to .757, so we can at least use these predictions for the submission and append these values to the hacked .csv file.

In [43]:
floating_df

,Cardiomegaly,Enlarged Cardiomediastinum,Fracture,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Support Devices,Id
0,0.0,0.0,0.0,0.000000,-0.565292,-0.661670,0.0,0.0,0.000000,18
1,0.0,0.0,0.0,0.000000,0.484359,-0.676549,0.0,0.0,0.000000,19
2,0.0,0.0,0.0,0.000000,-0.989416,0.000000,0.0,0.0,0.684321,44
3,0.0,0.0,0.0,0.554248,-0.994104,0.000000,0.0,0.0,0.765685,45
4,0.0,0.0,0.0,0.000000,-0.940837,0.000000,0.0,0.0,0.589707,57
...,...,...,...,...,...,...,...,...,...,...
22591,0.0,0.0,0.0,0.000000,-0.973348,0.000000,0.0,0.0,0.000000,223392
22592,0.0,0.0,0.0,0.507681,-0.994964,0.000000,0.0,0.0,0.745874,223394
22593,0.0,0.0,0.0,0.527522,-0.989828,0.000000,0.0,0.0,0.000000,223401
22594,0.0,0.0,0.0,0.000000,-0.983007,0.000000,0.0,0.0,0.589275,223403


In [44]:
pleural_effusion = floating_df['Pleural Effusion'].values

In [45]:
pleural_effusion.shape

(22596,)

In [46]:
hacked_file = pd.read_csv('sub_1_MSE_hack_file.csv')

In [47]:
hacked_file

,Unnamed: 0.1,Unnamed: 0,Cardiomegaly,Enlarged Cardiomediastinum,Fracture,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Support Devices,Id
0,0,0,0,-0.25,0.5,0.8,-0.75,-1,0.5,0,0.9,18
1,1,1,0,-0.25,0.5,0.8,-0.75,-1,0.5,0,0.9,19
2,2,2,0,-0.25,0.5,0.8,-0.75,0,0.5,0,0.9,44
3,3,3,0,-0.25,0.5,0.8,-0.75,0,0.5,0,0.9,45
4,4,4,0,-0.25,0.5,0.8,-0.75,0,0.5,0,0.9,57
...,...,...,...,...,...,...,...,...,...,...,...,...
22591,22591,22591,0,-0.25,0.5,0.8,-0.75,0,0.5,0,0.9,223392
22592,22592,22592,0,-0.25,0.5,0.8,-0.75,0,0.5,0,0.9,223394
22593,22593,22593,0,-0.25,0.5,0.8,-0.75,1,0.5,0,0.9,223401
22594,22594,22594,0,-0.25,0.5,0.8,-0.75,0,0.5,0,0.9,223403


In [48]:
hacked_file['Pleural Effusion'] = pleural_effusion

In [49]:
hacked_file

,Unnamed: 0.1,Unnamed: 0,Cardiomegaly,Enlarged Cardiomediastinum,Fracture,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Support Devices,Id
0,0,0,0,-0.25,0.5,0.8,-0.75,-0.661670,0.5,0,0.9,18
1,1,1,0,-0.25,0.5,0.8,-0.75,-0.676549,0.5,0,0.9,19
2,2,2,0,-0.25,0.5,0.8,-0.75,0.000000,0.5,0,0.9,44
3,3,3,0,-0.25,0.5,0.8,-0.75,0.000000,0.5,0,0.9,45
4,4,4,0,-0.25,0.5,0.8,-0.75,0.000000,0.5,0,0.9,57
...,...,...,...,...,...,...,...,...,...,...,...,...
22591,22591,22591,0,-0.25,0.5,0.8,-0.75,0.000000,0.5,0,0.9,223392
22592,22592,22592,0,-0.25,0.5,0.8,-0.75,0.000000,0.5,0,0.9,223394
22593,22593,22593,0,-0.25,0.5,0.8,-0.75,0.000000,0.5,0,0.9,223401
22594,22594,22594,0,-0.25,0.5,0.8,-0.75,0.000000,0.5,0,0.9,223403


In [50]:
hacked_file.to_csv('sub_1_MSE_hack_file_v2.csv')

<div class = 'alert alert-info'>

# 3 custom channels analysis

In [23]:
# hacked_file = pd.read_csv('sub_1_MSE_hack_file.csv')

Unfortunately there is no change to the results. 

I was hoping that adding in information to the input channels about edge detection would help with better accuracy for pathologies related to size of organs like enlarged cardio. 

I think the issue is that the information in each of the three channels is conglomerated for the ViT and it averages over the three channels when patching into 16x16 pieces. 

I will need to run a model most likely with train data that is only comprised of edge detection to get MSE below 1 for things like enlarged cardio and cardiomegaly.